In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install spuco
%pip install -q pytorch-metric-learning

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as transforms
import cv2
import torchvision.models as models
from pytorch_metric_learning import losses
import torch.optim as optim
from torch.optim import lr_scheduler
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
magnitude = "MEDIUM"

magnitude_dict = {
    "SMALL": SpuriousFeatureDifficulty.MAGNITUDE_SMALL,
    "MEDIUM":SpuriousFeatureDifficulty.MAGNITUDE_MEDIUM,
    "LARGE":SpuriousFeatureDifficulty.MAGNITUDE_LARGE,
}

class Config:
  spurious_correlation_strength  = 0.3
  spurious_feature_difficulty = magnitude_dict[magnitude]
  label_noise = 0.001
  size = 28
  flip_probability = 0.5
  batch_size = 256
  backbone_pretrained = False
  projection_dim = 128
  T_max = 10
  lr = 8e-4
  min_lr = 1e-5
  weight_decay = 1e-6
  temperature = 0.5

In [5]:
model_type = 'simCLR'
save_dir = f"/content/drive/MyDrive/CS260D_final_project/changing_spurious_correlation_strength/model_parameters/{Config.spurious_correlation_strength}_{magnitude}"
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)

# Setup Data


In [6]:
classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="train",
    label_noise=Config.label_noise
)
trainset.initialize()

valset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=Config.spurious_feature_difficulty,
    spurious_correlation_strength=Config.spurious_correlation_strength,
    classes=classes,
    split="test",
    label_noise=Config.label_noise
)
valset.initialize()


100%|██████████| 10000/10000 [00:01<00:00, 7151.75it/s]


In [7]:
# Show example from each group
from IPython.display import display
for (k, v) in trainset.group_partition.items():
  print(f"Group: {k}, Qty: {len(v)}")
  img = T.ToPILImage()(trainset[v[0]][0])
  display(img)

Group: (2, 3), Qty: 1570


Group: (0, 4), Qty: 1752


Group: (2, 0), Qty: 1619


Group: (0, 0), Qty: 3037


Group: (4, 0), Qty: 1633


...

Group: (0, 2), Qty: 1781


Group: (4, 2), Qty: 1679


Group: (4, 3), Qty: 1672


Group: (2, 4), Qty: 1571


Group: (1, 2), Qty: 1701


In [8]:

## Augmentation
from torchvision.transforms import v2

size = 28
train_transform = transforms.Compose([
    v2.ToImage(),
    v2.RandomResizedCrop(size),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
    v2.RandomGrayscale(p=0.2),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

test_transform = transforms.Compose([
    v2.ToImage(),
    v2.ToTensor(),
    v2.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:43: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


In [9]:
class SpucoDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img = self.dataset[index][0]
        label = self.dataset[index][1]

        if self.transform is None:
          return img, label

        augmented_img = self.transform(img)
        return (img, augmented_img), label

In [10]:
train_dataset = SpucoDataset(trainset, train_transform)
val_dataset = SpucoDataset(valset, test_transform)

dataloaders = {'train': DataLoader(dataset=train_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=True),
         'eval': DataLoader(dataset=val_dataset, batch_size=Config.batch_size, num_workers=2, pin_memory=True, shuffle=False)}

# Setup Model

In [11]:
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#ntxentloss
## https://kevinmusgrave.github.io/pytorch-metric-learning/losses/#selfsupervisedloss
criterion = losses.NTXentLoss(temperature=Config.temperature)
criterion = losses.SelfSupervisedLoss(criterion)

In [12]:
## https://github.com/berfukaraca/SSL-Cifar10-ContrastiveLearning/blob/main/colab_notebooks/SSL_SimCLR_Cifar10_CL.ipynb

class SimCLR_model(nn.Module):
    def __init__(self, base_encoder, projection_dim, n_features):
        super(SimCLR_model, self).__init__()
        self.n_features = n_features
        self.base_encoder = base_encoder

        self.projectionHead = nn.Sequential(
            nn.Linear(self.n_features, self.n_features, bias=False),
            nn.BatchNorm1d(self.n_features),
            nn.ReLU(), #non-linearity
            nn.Linear(self.n_features, projection_dim, bias=False),
            nn.BatchNorm1d(projection_dim),
        )

    def forward(self, x_i):
        out = self.projectionHead(self.base_encoder(x_i))

        return out

In [13]:
## model and optimizer
model = models.resnet50()
# print(model)
clr_model = SimCLR_model(model, Config.projection_dim, 1000) ## output dimension of resnet18 is 1000
optimizer = optim.Adam(clr_model.parameters(), lr=Config.lr, weight_decay=Config.weight_decay)
## cosine annealing lr: https://arxiv.org/pdf/1608.03983.pdf
scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=Config.T_max, eta_min=Config.min_lr)

clr_model.to(device)

SimCLR_model(
  (base_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [14]:
## Load the pre-trained model parameters

# checkpoints_path = f"/content/drive/MyDrive/CS260D_final_project/changing_spurious_correlation_strength/model_parameters/{Config.spurious_correlation_strength}_{magnitude}/"
checkpoints_path = f"/content/drive/MyDrive/CS260D_final_project/model_parameters/"
# find model
model_path = os.path.join(checkpoints_path, f'paramsend__bs_restart_r_{Config.spurious_correlation_strength}_{magnitude}.pt')

checkpoint = torch.load(model_path, map_location = torch.device(device))
loss= checkpoint['loss']
epoch = checkpoint['epoch']

clr_model.load_state_dict(checkpoint['net'])

<All keys matched successfully>

In [15]:
clr_model.to(device)

SimCLR_model(
  (base_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(


In [16]:
import os
from tqdm.notebook import tqdm
def train(epoch):
    print('\nEpoch: %d' % epoch)
    clr_model.train()
    s=0
    tot_loss = 0
    total = 0
    batch_idx=0
    i=0
    batch_num =1
    val_loss = 0
    for batch_idx, (images, _) in tqdm(enumerate(dataloaders['train'])):
        batch_tot_loss = 0
        images, augmented_images = images[0], images[1]
        images, augmented_images = images.to(device), augmented_images.to(device)
        optimizer.zero_grad()

        images_emb = clr_model(images)
        augmented_images_emb = clr_model(augmented_images)
        loss = criterion(images_emb, augmented_images_emb)

        loss.backward()

        optimizer.step()
        tot_loss += loss.item()
        batch_tot_loss += loss.item()
    scheduler.step()



    # Validation
    clr_model.eval()
    with torch.no_grad():
        for batch_idx, (images, _) in enumerate(dataloaders['eval']):
            val_batch_loss = 0

            images, augmented_images = images[0], images[1]
            images, augmented_images = images.to(device), augmented_images.to(device)


            images_emb = clr_model(images)
            augmented_images_emb = clr_model(augmented_images)
            loss = criterion(images_emb, augmented_images_emb)



            val_loss += loss.item()
            val_batch_loss += loss.item()

    print(f"epoch {epoch}train loss= {tot_loss/len(dataloaders['train'])}")
    print(f"epoch {epoch}val loss= {val_loss/len(dataloaders['train'])}")

    if epoch % 10 == 0:
        modelstate_save_name = f'modelstate_bs_restart_r_epoch{epoch}.pt'
        path_modelstate = os.path.join(save_dir, modelstate_save_name)
        torch.save(clr_model.state_dict(), path_modelstate)

    parameters_end = {
        'net': clr_model.state_dict(),
        'epoch': epoch,
        'loss': tot_loss/len(dataloaders['train'])
    }
    paramsend_save_name = 'paramsend_bs_restart_r.pt'
    path_paramsend = os.path.join(save_dir, paramsend_save_name)
    torch.save(parameters_end, path_paramsend)

In [17]:
# Check Loss
loss/len(dataloaders['train'])


6.16719142173199

In [18]:
# Continue training
for epoch in range(1,21):
  train(epoch)


Epoch: 1


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

0it [00:00, ?it/s]

epoch 1train loss= 5.4688445557939245
epoch 1val loss= 1.6664326241675844

Epoch: 2


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

...

0it [00:00, ?it/s]

epoch 19train loss= 4.736509168401677
epoch 19val loss= 1.523769368516638

Epoch: 20


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across th

0it [00:00, ?it/s]

epoch 20train loss= 4.728765759062259
epoch 20val loss= 1.506754570818962


In [19]:
scheduler.get_last_lr()

[0.0008000000000000034]